In [ ]:
from aocd import get_data
import functools

# get data
data = get_data(day=5, year=2023)
data = data.split("\n\n")

# process the seeds input for use by both parts
seeds = list(map(int, data[0].split(": ")[1].split(" ")))

# process the maps into lists of ints used by both parts
for index, map_in in enumerate(data):
    if index > 0:
        range_spec = map_in.split(":\n")[1].split("\n")
        
range_specs = [x.split(":\n")[1].split("\n") for x in data[1:]]
range_specs_ints = [[list(map(int, x.split(" "))) for x in suba] for suba in range_specs]


In [ ]:

def track_map(map_in, seed_in):
    dest = seed_in
    for x in map_in:
        if x[1] <= seed_in <= (x[1] + x[2] - 1):
            dest = x[0] + (seed_in - x[1])
    return(dest)
    
def track_all(seed_in):
    for x in range_specs_ints:
        seed_in = track_map(x, seed_in)
    return(seed_in)
    
print("Part 1:")
print(min([track_all(x) for x in seeds]))

In [ ]:

# seeds are now ranges of seeds
seeds_range_start = seeds[::2]
seeds_range_width = seeds[1::2]

seeds_expanded = [range(x, x+y) for x, y in zip(seeds_range_start, seeds_range_width)]
#print(seeds_expanded)

# this turns the map ranges into a new format, which is the range of the input
# and the offset which should be applied to seeds in that input range:
# [range(start, stop), offset]
# it also creates additional ranges with offset 0 so that the whole range of
# seeds is mapped to something
range_specs_ranges = []
for xx in range_specs_ints:
    ranges_mapped = [[range(y, y+z), x-y] for x, y, z in xx]
    
    starts = [y for x, y, z in xx]
    starts = sorted(starts)
    starts.append(int(1e8))

    ends = [y+z for x, y, z in xx]
    ends = sorted(ends)
    ends.insert(0,0)
    
    ranges_unmapped = []
    for s,e in zip(starts, ends):
        if s != e:
            ranges_unmapped.append([range(e, s), 0])
        
    ranges_mapped_and_unmapped = ranges_mapped + ranges_unmapped
    range_specs_ranges.append(ranges_mapped_and_unmapped)
    
#print(range_specs_ranges)

# finds the intersect of two ranges (thanks SO)
def range_intersect(r2, r1):
    return range(max(r1.start,r2.start), min(r1.stop,r2.stop)) or None

# this loops through the seed ranges and map ranges to output the
# new ranges of the seeds for the next stage
def range_mapper(map_range_in, seed_range_in):
    
    dest_range = []
    for seed_range in seed_range_in:
        for map_range in map_range_in:
            overlap = range_intersect(map_range[0],seed_range)
            if overlap:
                dest_range.append(range(overlap.start + map_range[1], overlap.stop + map_range[1]))

    return dest_range

# iterates through the map stages
def track_all_ranges(seed_in):
    for x in range_specs_ranges:
        seed_in = range_mapper(x, seed_in)
    return(seed_in)

print("Part 2:")
# find the lowest end of all the resulting ranges
print(min([x.start for x in track_all_ranges(seeds_expanded)]))